In [1]:
import pandas as pd
import re
import json

# data cleaning for resumes.

* a. Trim and clean the text of special characters.
* b. Using breaks, commas or spaces, attempt to separate the annotations into individual skills

## Read data

In [34]:
resume_data = pd.read_json("Entity Recognition in Resumes.json", lines=True, orient="records")
resume_data.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [44]:
resume_data = resume_data.dropna(axis=1).dropna(axis=1)

In [20]:
resume_data["annotation"][0][:2]

[{'label': ['Skills'],
  'points': [{'start': 1295,
    'end': 1621,
    'text': '\n• Programming language: C, C++, Java\n• Oracle PeopleSoft\n• Internet Of Things\n• Machine Learning\n• Database Management System\n• Computer Networks\n• Operating System worked on: Linux, Windows, Mac\n\nNon - Technical Skills\n\n• Honest and Hard-Working\n• Tolerant and Flexible to Different Situations\n• Polite and Calm\n• Team-Player'}]},
 {'label': ['Skills'],
  'points': [{'start': 993,
    'end': 1153,
    'text': 'C (Less than 1 year), Database (Less than 1 year), Database Management (Less than 1 year),\nDatabase Management System (Less than 1 year), Java (Less than 1 year)'}]}]

In [22]:
txt = resume_data["content"][0]
txt

"Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\n\

In [23]:
txt[1295:1622]

'\n• Programming language: C, C++, Java\n• Oracle PeopleSoft\n• Internet Of Things\n• Machine Learning\n• Database Management System\n• Computer Networks\n• Operating System worked on: Linux, Windows, Mac\n\nNon - Technical Skills\n\n• Honest and Hard-Working\n• Tolerant and Flexible to Different Situations\n• Polite and Calm\n• Team-Player'

## a. Trim and clean the text of special characters.

In [32]:
def clean_text(txt):
    "Trim and clean the text of special characters"
    txt = re.sub("[^a-zA-Z0-9]+", " ", txt)
    txt = re.sub("\s+", " ", txt)
    txt = txt.strip()
    return txt


In [33]:
clean_text(txt)

'Abhishek Jha Application Development Associate Accenture Bengaluru Karnataka Email me on Indeed indeed com r Abhishek Jha 10e7a8cb732bc43a To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company s growth in best possible ways Willing to relocate to Bangalore Karnataka WORK EXPERIENCE Application Development Associate Accenture November 2017 to Present Role Currently working on Chat bot Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input Also Training the bot for different possible utterances Both positive and negative which will be given as input by the user EDUCATION B E in Information science and engineering B v b college of engineering and technology Hubli Karnataka August 2013 to June 2017 12th in Mathematics Woodbine modern school April 2011 to March 2013 10th Kendriya Vidyalaya April 2001 to March 2011 SKILLS C Less than 1 year Database Less than 1 year Databa